In [1]:
import time
import argparse
import os.path

import numpy as np
import scipy.sparse
import pandas as pd
import networkx as nx

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

!export DGLBACKEND=pytorch
import dgl
import dgl.multiprocessing as mp
import dgl.function as fn
import dgl.nn.pytorch as dglnn

import tqdm

Using backend: pytorch


In [2]:
device = default_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('CUDA version:', torch.version.cuda)
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

GeForce RTX 2080 Ti
CUDA version: 10.2
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
node_features = 16

In [4]:
if os.path.isfile('dataset/paper_author_relationship.bin'):
    graph = dgl.load_graphs('dataset/paper_author_relationship.bin')[0][0]
else:
    data_authors, data_papers = [], []
    with open('dataset/paper_author_relationship.csv') as f:
        for i, line in enumerate(f):
            paper = i
            authors = map(int, line.split(','))
            for author in authors:
                data_authors.append(author)
                data_papers.append(paper)
    graph = dgl.heterograph({
        ('author', 'writes', 'paper'): (data_authors, data_papers),
        ('paper', 'written-by', 'author'): (data_papers, data_authors)
    }, device=device)
    dgl.save_graphs('dataset/paper_author_relationship.bin', [graph])
    del data_authors, data_papers
graph

Graph(num_nodes={'author': 2293783, 'paper': 449006},
      num_edges={('author', 'writes', 'paper'): 1142106, ('paper', 'written-by', 'author'): 1142106},
      metagraph=[('author', 'paper', 'writes'), ('paper', 'author', 'written-by')])

In [5]:
train = pd.read_csv(
    'dataset/train_dataset.csv',
    header=0,
    names=['source', 'target', 'label'],
    dtype={'label': bool},
    sep=',\s+',
    engine='python'
)
val = pd.read_csv(
    'dataset/valid_dataset.csv',
    header=0,
    names=['source', 'target', 'label'],
    dtype={'label': bool},
    sep=',\s+',
    engine='python'
)
query = pd.read_csv(
    'dataset/query_dataset.csv',
    header=0,
    names=['source', 'target'],
    dtype={'label': bool},
    sep=',\s+',
    engine='python'
)

In [6]:
class GCN(nn.Module):
    def __init__(self, graph, in_features, hidden_features, out_features, rel_names):
        super(GCN, self).__init__()

        embed_dict = {
            ntype: nn.Parameter(torch.Tensor(
                graph.number_of_nodes(ntype), in_features))
            for ntype in graph.ntypes
        }

        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(
                embed, gain=nn.init.calculate_gain('leaky_relu', 0.05))

        self.embed = nn.ParameterDict(embed_dict)

        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_features, hidden_features)
            for rel in rel_names
        })

        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hidden_features, out_features)
            for rel in rel_names
        })

    def forward(self, blocks, x):
        x = F.leaky_relu(self.conv1(blocks[0], x), 0.05)
        x = F.tanh(self.conv2(blocks[1], x))
        return x

In [7]:
class Generator(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(Generator, self).__init__()
        self.linear1 = nn.Linear(in_features, hidden_features)
        self.linear2 = nn.Linear(hidden_features, out_features)

    def forward(self, z):
        z = F.leaky_relu(self.linear1(z), 0.05)
        z = F.leaky_relu(self.linear2(z), 0.05)
        return z


class Discriminator(nn.Module):
    def __init__(self, num_features, hidden_features):
        super(Discriminator, self).__init__()
        self.linear = nn.Linear(num_features, hidden_features)
        self.classifier = nn.Linear(hidden_features, 1)

    def forward(self, x):
        x = F.leaky_relu(0.05, self.linear(x))
        x = self.classifier(x)
        return x

In [8]:
node_features = 16
hidden_features = 2 * node_features


sampler = dgl.dataloading.MultiLayerNeighborSampler([
    {('author', 'writes', 'paper'): 5},
    {('paper', 'written-by', 'author'): 5},
])
dataloader = dgl.dataloading.NodeDataLoader(
    graph, graph.nodes['author'], sampler,
    batch_size=1024, shuffle=True, drop_last=False,
    device=device,
)

model = GCN(node_features, hidden_features, node_features, graph.etypes).to(device)
optimizer = optim.Adam(model.parameters())


G = Generator(node_features, hidden_features, node_features).to(device)
Du = Discriminator(node_features, hidden_features).to(device)
Db = Discriminator(node_features, hidden_features).to(device)

Lu = nn.BCEWithLogitsLoss().to(device)
Lb = nn.BCEWithLogitsLoss().to(device)

optimizer_G = optim.Adam(G.parameters(), lr=3e-4, betas=(0.5, 0.99))
optimizer_Du = optim.Adam(Du.parameters(), lr=3e-4, betas=(0.5, 0.99))
optimizer_Db = optim.Adam(Db.parameters(), lr=3e-4, betas=(0.5, 0.99))


AssertionError: nids should be a dict of node type and ids for graph with multiple node types

In [ ]:
# for input_nodes, output_nodes, blocks in dataloader:
#     optimizer.zero_grad()
#     blocks = [b.to(torch.device(device)) for b in blocks]
#     input_features = blocks[0].srcdata['features']
#     output_predictions = model(blocks, input_features)
#     output_labels = blocks[-1].dstdata['label']
#     loss = compute_loss(output_predictions, output_labels)
#     loss.backward()
#     optimizer.step()